In [3]:
import math

In [4]:
class Scaler:

  def __init__(self, data, children=(), op=None):
    self.data = data
    self._prev = set(children)
    self._op = op
    self.grad = 0
    self._backward = lambda: None


  def __add__(self, other):
    out = Scaler(self.data + other.data, children=(self, other), op='+')
    def _backward():
      self.grad += out.grad
      other.grad += out.grad
    out._backward = _backward
    return out

  def __mul__(self, other):
    out = Scaler(self.data * other.data, children=(self, other), op='x')
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
    return out

  def tanh(self):
    x = self.data
    th = (math.exp(2*x) -1)/(math.exp(2*x) + 1)
    out = Scaler(th, (self,), 'tanh')
    def _backward():
      self.grad += (1-th**2) * out.grad
    out._backward = _backward
    return out

  def backward(self):

    topo = []
    visited = set()
    def build_topo(v):
        if v not in visited:
            visited.add(v)
            for child in v._prev:
                build_topo(child)
            topo.append(v)
    build_topo(self)

    self.grad = 1
    for v in reversed(topo):
        v._backward()

  def __str__(self):
    return f'Scaler {self.data}'

  def __repr__(self):
    return f'Scaler {self.data}'

